# C# Job System 

## Cos'è
Sistema di Unity per scrivere codice **multithreaded sicuro e semplice** che migliora le performance dei giochi.

## Vantaggi principali
- **Performance**: aumento significativo dei frame rate
- **Efficienza energetica**: riduzione consumo batteria (mobile) grazie al Burst compiler
- **Integrazione nativa**: condivide i worker threads con Unity, evitando la creazione eccessiva di thread che competerebbero per le risorse CPU

## Caratteristiche chiave
- Si integra con ECS (Entity Component System)
- Genera codice macchina efficiente per tutte le piattaforme
- Facilita la scrittura di codice corretto in ambiente multithreaded

## Concetto fondamentale
Il Job System di Unity **coopera** con il sistema nativo interno di Unity, ottimizzando l'uso dei core CPU disponibili senza sprechi.

---

# Multithreading

## Concetto Base
1. **Single-thread**: un'istruzione alla volta, esecuzione sequenziale.
2. **Multithreading**: più istruzioni eseguono **simultaneamente** su più core CPU.

## Come Funziona
- **Main thread**: thread principale che crea altri thread
- **Thread paralleli**: lavorano contemporaneamente e sincronizzano i risultati con il main thread

## Il Problema nel Game Development
I giochi hanno **molte piccole istruzioni** invece di pochi task lunghi:
- Creare un thread per ogni task = troppi thread
- Troppi thread = **contention** (competizione per risorse CPU)
- Conseguenza: **context switching** frequente (salvare/ripristinare stato del thread) = **costoso in performance**

## Soluzione: Thread Pool
Pool di thread riutilizzabili, ma il rischio rimane se hai più thread che core CPU.

## Tipi di Multithreading (da Wikipedia)

### 1. **Block/Coarse-grained** (più semplice)
- Switch thread quando uno si blocca (es. cache miss)
- Simile al multitasking cooperativo
- Richiede registri duplicati hardware

### 2. **Fine-grained/Interleaved**
- Switch ogni ciclo CPU (1 istruzione per thread)
- Elimina data dependency stalls
- Costo: pipeline deve tracciare thread ID

### 3. **Simultaneous (SMT)** (più avanzato)
- Più istruzioni da **più thread** nello **stesso ciclo**
- Esempio: Intel Hyper-Threading
- Sfrutta parallelismo tra thread diversi

## Pro e Contro

**Vantaggi**:
- ✓ Throughput migliore (lavoro totale completato)
- ✓ Uso efficiente risorse (se un thread è in stallo, altri lavorano)

**Svantaggi**:
- ✗ Thread competono per cache/TLB
- ✗ Overhead di sincronizzazione (anche 100x più costoso)
- ✗ Code ottimizzato (assembly, MMX) può peggiorare
- ✗ Complessità software (app e OS devono cambiare)

---

# Job System 

## Definizione
Sistema che gestisce codice multithreaded creando **jobs** invece di thread diretti.

## Componenti Chiave

### 1. **Worker Threads**
- Pool di thread gestiti dal sistema
- **1 worker thread = 1 core CPU logico** (evita context switching)
- Alcuni core riservati per OS/altre app

### 2. **Job Queue**
- Coda dove vengono inseriti i jobs
- Worker threads prendono jobs dalla coda ed eseguono

### 3. **Dependency Management**
- Sistema garantisce ordine di esecuzione corretto
- Gestisce automaticamente le dipendenze tra jobs

## Cos'è un Job?

**Piccola unità di lavoro** che esegue un task specifico:
- Riceve parametri
- Opera sui dati
- Funziona come una method call

**Due tipi**:
- **Self-contained**: indipendente
- **Dependent**: richiede completamento di altri jobs

## Job Dependencies

### Esempio Pratico
```
jobA dipende da jobB
→ jobB deve completare PRIMA che jobA inizi
```

**Perché servono**: Nel game dev i jobs raramente sono isolati:
- Job 1: calcola posizioni nemici
- Job 2: usa quelle posizioni per pathfinding
- Job 3: applica movimento basato su pathfinding

Il sistema garantisce: Job 1 → Job 2 → Job 3

## Differenza Cruciale: Jobs vs Threads

| **Thread tradizionale** | **Job System** |
|------------------------|----------------|
| Crei/distruggi thread per ogni task | Riusi worker threads esistenti |
| Molti thread = context switching | Thread fissi = no context switching |
| Gestisci manualmente sincronizzazione | Sistema gestisce dependencies automaticamente |

---

[The safety system in the C# Job System](https://docs.unity3d.com/2020.1/Documentation/Manual/JobSystemSafetySystem.html) --> Da fare
